Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. Получившийся скрипт необходимо выложить в гит репозиторий и прислать ссылку в качестве результата прохождения тестового задания. Данные выкладывать в гит не следует. 

Главные задачи, которые должен выполнять скрипт:
+ Извлекать реплики с приветствием – где менеджер поздоровался. 
+ Извлекать реплики, где менеджер представил себя. 
+ Извлекать имя менеджера. 
+ Извлекать название компании. 
+ Извлекать реплики, где менеджер попрощался.
+ Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

Рекомендации:
+ Сделать локальную копию файла test_data.csv, в исходнике никакие данные не менять!
+ Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.
+ Для выполнения задачи можно использовать любые библиотеки и NLP модели. 
+ Попробуйте учесть возможные синонимичные выражения, которые могут помочь с извлечением данных сущностей. 


In [1]:
import pandas as pd
df = pd.read_csv('test_data.csv')

In [2]:
# Расширим датасет несколькими примерами для теста

df.loc[len(df)] = [6, 0, 'manager', 'меня зовут ангелина васильевна я представляю компанию золотое яблоко есть ли у вас минута свободного времени']
df.loc[len(df)] = [7, 0, 'manager', 'меня зовут иванова анастасия компания ла мода у нас есть информация о вашем заказе']
df.loc[len(df)] = [8, 0, 'manager', 'меня зовут петрова екатерина я из компании мвидео у нас есть информация о вашем заказе']
df.text = df.text.str.lower()
df

,dlg_id,line_n,role,text
0,0,0,client,алло
1,0,1,manager,алло здравствуйте
2,0,2,client,добрый день
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,ага
...,...,...,...,...
478,5,141,client,да да тогда созвонимся ага спасибо вам давайте
479,5,142,manager,ну до свидания хорошего вечера
480,6,0,manager,меня зовут ангелина васильевна я представляю к...
481,7,0,manager,меня зовут иванова анастасия компания ла мода ...


Видим что датасет весь в lowercase (кроме первых букв в строке). Найти подходящую caseless модель для русского языка не удалось, зато из статьи на хабре (https://habr.com/ru/post/581960/) нашел модель для расстановки пунктуации и заглавных букв в тексте. Это позволит использовать предобученные NER модели.

In [3]:
# Загружаем модель из статьи с Хабра
import os
import yaml
import torch
from torch import package
torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)

with open('latest_silero_models.yml', 'r') as yaml_file:
    models = yaml.load(yaml_file, Loader=yaml.SafeLoader)
model_conf = models.get('te_models').get('latest')

model_url = model_conf.get('package')

model_dir = "downloaded_model"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, os.path.basename(model_url))
if not os.path.isfile(model_path):
    torch.hub.download_url_to_file(model_url,
                                   model_path,
                                   progress=True)


imp = package.PackageImporter(model_path)
model = imp.load_pickle("te_model", "model")

def apply_te(text, lan='en'):
    return model.enhance_text(text, lan)

/home/michael/miniconda3/envs/bewise2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df['text'] = df['text'].str.replace('+', 'плюс')
df['punct_text'] = df['text'].apply(apply_te)

/home/michael/miniconda3/envs/bewise2/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [5]:
greetings = ['здравствуйте', 'добрый день', 'доброе утро', 'добрый вечер']
df['has_greeting'] = df['text'].apply(lambda sent: any(g in sent for g in greetings))

goodbye_list = ['до свидания', 'всего доброго', 'всего хорошего']
df['has_goodbye'] = df['text'].apply(lambda sent: any(g in sent for g in goodbye_list))

In [6]:
# Извлекаем реплики с приветствием – где менеджер поздоровался. 
df[(df['has_greeting']) & (df['role']=='manager')]

,dlg_id,line_n,role,text,punct_text,has_greeting,has_goodbye
1,0,1,manager,алло здравствуйте,Алло здравствуйте!,True,False
110,1,1,manager,алло здравствуйте,Алло здравствуйте!,True,False
166,2,2,manager,алло здравствуйте,Алло здравствуйте!,True,False
250,3,1,manager,алло дмитрий добрый день,Алло Дмитрий добрый день.,True,False


In [7]:
# Извлекаем реплики, где менеджер попрощался.
df[(df['has_goodbye']) & (df['role']=='manager')]

,dlg_id,line_n,role,text,punct_text,has_greeting,has_goodbye
108,0,108,manager,всего хорошего до свидания,Всего хорошего до свидания.,False,True
162,1,53,manager,угу да вижу я эту почту хорошо тогда исправлю ...,Угу! Да вижу я эту почту хорошо тогда исправлю...,False,True
163,1,54,manager,до свидания,До свидания.,False,True
300,3,51,manager,угу все хорошо да понедельника тогда всего доб...,Угу все хорошо да понедельника тогда всего доб...,False,True
335,4,33,manager,во вторник все ну с вами да тогда до вторника ...,Во вторник все ну с вами! Да тогда до вторника...,False,True
479,5,142,manager,ну до свидания хорошего вечера,Ну до свидания хорошего вечера?,False,True


### Ищем имя сотрудника
Теперь можно легко найти имя менеджера из punct_text. Возьмем NER модель slovnet, так как она достаточно легкая чтобы работать на CPU, но имеет метрики только на 1-2% хуже чем deeppavlov.

Затем с ищем с помощью нее имена в текстах с расставленной пунктуацией. Если это текст менеджера и это только начало разговора (первые 6 реплик), то считаем что это имя менеджера. Чтобы не зацеплять другие слова сделаем стоп-лист. А чтобы обрабатывать сложные имена в формате ФИ, ФИО, идем циклом.

In [8]:
from navec import Navec
from slovnet import NER
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re 


navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)


not_names = stopwords.words("russian")
not_names.extend(['угу', 'алло', 'да', 'не', 'нет', 'это', 'я'])
not_names = set(not_names)


def find_ner_info_in_text(punct_text):
    markup = ner(punct_text)
    return markup.spans


def find_name(row):
    name = []
    name_idx = -1
    if row['line_n'] < 7:
        for span in row['ner_info']:
            if span.type=='PER' :
                if name_idx == -1 or span.start - name_idx < 4 :
                    name_word = row['punct_text'][span.start : span.stop]
                    name_words = word_tokenize(name_word)
                    for word in name_words:
                        if word.lower() not in not_names:
                            name.append(word)
                            if name_idx < span.start:
                                name_idx = span.start + len(word)
                            else: 
                                name_idx+=len(word)
                else:
                    break
    return ' '.join(name) if len(name) > 0 else np.nan


def find_manager_name_in_dialogue(df_dialogue):
    possible_name_announcements = [r'меня (\w*) зовут', r'меня зовут (\w*)']
    compiled_regex = re.compile("(%s|%s)" % (possible_name_announcements[0], possible_name_announcements[1]), re.IGNORECASE)

    re_names = df_dialogue[df_dialogue['role']=='manager']['punct_text'].str.extractall(compiled_regex)
    re_name = None
    if len(re_names) > 0:
        re_names = re_names.iloc[0]
        re_names = re_names.drop(0)
        re_name = re_names.dropna().iloc[0]


    ner_names = df_dialogue[(df_dialogue['role']=='manager') & (df_dialogue['ner_info'].notna())].apply(find_name, axis=1).dropna()

    if re_name is not None:
        if len(ner_names) > 0:
            for name in ner_names:
                if re_name in name:
                    return name
        return re_name
        
    if len(ner_names) > 0:
        return ner_names.iloc[0]
    
    return None


df['ner_info'] = df['punct_text'].apply(find_ner_info_in_text)

for i in range(9):
    print(f'Dialogue {i}:')
    manager_name = find_manager_name_in_dialogue(df[df['dlg_id'] == i])
    if manager_name is not None:
        manager_speech = df[(df['dlg_id'] == i) & (df['role'] == 'manager') & (df['text'].str.contains(manager_name.lower()))].text.iloc[0]
        print('Name: ', manager_name)
        print('Speech: ', manager_speech)
    else:
        print('No name')
    print()
    
# диалоги 6-8 добавлены для теста.

Dialogue 0:
Name:  Ангелина
Speech:  меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается

Dialogue 1:
Name:  Ангелина
Speech:  меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается

Dialogue 2:
Name:  Ангелина
Speech:  меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там

Dialogue 3:
Name:  Максим
Speech:  добрый меня максим зовут компания китобизнес удобно говорить

Dialogue 4:
No name

Dialogue 5:
Name:  Анастасия
Speech:  да это анастасия

Dialogue 6:
Name:  Ангелина Васильевна
Speech:  меня зовут ангелина васильевна я представляю компанию золотое яблоко есть ли у вас минута свободного времени

Dialogue 7:
Name:  Иванова Анастасия
Speech:  меня зовут иванова анастасия компания ла мода у нас есть информация о вашем заказе

Dialogue 8:
Name:  Петрова Екатер

### Ищем название компании

Считаем, что название компании может быть произнесено только в начале диалога после слова "компания". Для этого ищем слова, идущие после "компании" до следующего знака препинания. После этого с помощью mystem опредляем части речи слов. Если это существительное, числительное или прилагательное, берем их в название. Сделано это для того, чтобы избежать попадания глаголов и наречий в названия.

P.S. не удалось вычленить синтаксическую связь из текста, чтобы определять границы. NER также не видит в названиях ORG.

In [9]:
from pymystem3 import Mystem
mystem = Mystem()

def is_word_a_part_of_name(word):
    pos = mystem.analyze(word)
    pos = pos[0]['analysis'][0]['gr']
    pos = pos.replace('=', ',')
    pos = pos.split(',')[0]
    return pos in ['S', 'CONJ', 'A', 'ANUM', 'COM', 'NUM']
    


def get_company_name(df_dialogue):
    company_announcements = [r'компания\s?\W? ([\w\s]+)', r'представляю компанию\s?\W? ([\w\s]+)', r'из компании\s?\W? ([\w\s]+)']
    compiled_regex = re.compile("(%s|%s|%s)" % (company_announcements[0], company_announcements[1], company_announcements[2]), re.IGNORECASE)
    re_names = df_dialogue[df_dialogue['role']=='manager']['punct_text'].str.extractall(compiled_regex)
    
    name = []

    if len(re_names) > 0:
        re_names=re_names.iloc[0]
        re_names = re_names.drop(0)
        re_name = re_names.dropna().iloc[0] 
        
        for word in word_tokenize(re_name):
            if len(name) == 0 or is_word_a_part_of_name(word):
                name.append(word)
            else:
                break
    return ' '.join(name) if len(name) > 0 else None



for i in range(9):
    print(f'Dialogue {i}:', get_company_name(df[df['dlg_id'] == i]))
# диалоги 6-8 добавлены для теста.

Dialogue 0: диджитал бизнес
Dialogue 1: диджитал бизнес
Dialogue 2: диджитал бизнес
Dialogue 3: Китобизнес
Dialogue 4: None
Dialogue 5: None
Dialogue 6: Золотое яблоко
Dialogue 7: ла мода
Dialogue 8: Мвидео


### Проверка условий

In [10]:
# Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»
dialogues_greetings = df[df['role'] == 'manager'].groupby('dlg_id')['has_greeting'].any()
dialogues_goodyes = df[df['role'] == 'manager'].groupby('dlg_id')['has_goodbye'].any()
dialogue_condition = dialogues_greetings & dialogues_goodyes
print(dialogue_condition)

# диалоги 6-8 добавлены для теста.

dlg_id
0     True
1     True
2    False
3     True
4    False
5    False
6    False
7    False
8    False
dtype: bool
